In [3]:
import sys

import torch

sys.path.insert(0, '/home/claranunesbarrancos/XAI/CN/peepholelib/')
print("sys.path:", sys.path)

# python stuff

from pathlib import Path as Path
from peepholelib.datasets.cifar import Cifar
from peepholelib.datasets.transforms import mobile_netv2 as mobile_net_transform
from peepholelib.models.model_wrap import ModelWrap

import torchvision
from torchvision import transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from cuda_selector import auto_cuda
import torch.nn as nn
import torch.optim as optim

sys.path: ['/home/claranunesbarrancos/XAI/CN/peepholelib/', '/srv/newpenny/XAI/CN/XAI/src', '/srv/newpenny/conda/envs/xai-venv/lib/python312.zip', '/srv/newpenny/conda/envs/xai-venv/lib/python3.12', '/srv/newpenny/conda/envs/xai-venv/lib/python3.12/lib-dynload', '', '/srv/newpenny/conda/envs/xai-venv/lib/python3.12/site-packages', '/srv/newpenny/conda/envs/xai-venv/lib/python3.12/site-packages/FALCONN-1.4.0-py3.12-linux-x86_64.egg', '/srv/newpenny/conda/envs/xai-venv/lib/python3.12/site-packages/setuptools/_vendor']


In [4]:
use_cuda = torch.cuda.is_available()
cuda_index = torch.cuda.device_count() - 1
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(f"Using {device} device")

Using cuda:0 device


## Dataset

In [5]:
from torch.utils.data import DataLoader, random_split

dataset = 'CIFAR100' 
seed = 29
bs = 96 
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(degrees=15),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3))

])

ds_path = '/srv/newpenny/dataset/CIFAR100'

cifar_dataset = torchvision.datasets.__dict__[dataset](root=ds_path, 
                                                       train=True, 
                                                       transform=preprocess, 
                                                       download=True)
print(f"Total number of images in CIFAR-100 training dataset: {len(cifar_dataset)}")

# split the dataset into train and validation sets
train_size = int(0.8 * len(cifar_dataset))
val_size = len(cifar_dataset) - train_size
generator = torch.Generator().manual_seed(seed)
train_dataset, val_dataset = random_split(cifar_dataset, [train_size, val_size], generator=generator)
test_dataset = torchvision.datasets.__dict__[dataset](root=ds_path, 
                                                      train=False, 
                                                      download=True, 
                                                      transform=preprocess)

# create DataLoaders
num_workers = 16

train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

Files already downloaded and verified
Total number of images in CIFAR-100 training dataset: 50000
Files already downloaded and verified


## Model

In [6]:
model_dir = '/srv/newpenny/XAI/models'
model_name = 'mobilenet_v2'
model_path = 'CN_model=mobilenet_v2_dataset=CIFAR100_optim=Adam_scheduler=RoP_lr=0.001_factor=0.1_patience=5.pth'
model = torchvision.models.mobilenet_v2(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(model.last_channel, 100)
)

model = model.to(device)
print(f"Model architecture:\n{model}")
initial_weights = model.features[0][0].weight.clone()
initial_weights_range = initial_weights.data.cpu().numpy()
print(f'Initial weights range: min={initial_weights_range.min()}, max={initial_weights_range.max()}')


/srv/newpenny/conda/envs/xai-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/srv/newpenny/conda/envs/xai-venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model architecture:
MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): 

In [ ]:
'''
#nn_mobilenet = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)
#model = torchvision.models.mobilenet_v2(pretrained=False)
model = torchvision.models.__dict__[model_name](weights=None)
initial_weights = model.features[0][0].weight.clone()
initial_weights_range = initial_weights.data.cpu().numpy()
print(f'Initial weights range: min={initial_weights_range.min()}, max={initial_weights_range.max()}')
model.classifier[1] = torch.nn.Linear(model.last_channel, 100)
#model.load_state_dict(torch.load(model_path, map_location=device)) # load saved parameters (weights and biases)
model = model.to(device)
trained_weights = model.features[0][0].weight.data.cpu().numpy()
print(f'Trained weights range: min={trained_weights.min()}, max={trained_weights.max()}')
'''

Initial weights range: min=-0.26537197828292847, max=0.23494365811347961
Trained weights range: min=-0.26537197828292847, max=0.23494365811347961


## Training

In [7]:
def accuracy(outputs, targets):
    _, predicted = torch.max(outputs, 1)  # get the class index with the highest probability
    correct = (predicted == targets).sum().item()
    total = targets.size(0)
    return correct / total

In [8]:
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler


def train_model(model, train_loader, val_loader, device, num_epochs=150):
    best_val_accuracy = 0
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, min_lr=1e-6)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for data in tqdm(train_loader):
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()        
            
            correct_predictions += accuracy(outputs, targets) * targets.size(0)
            total_predictions += targets.size(0)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct_predictions = 0
        val_total_predictions = 0

        with torch.no_grad():
            for data in val_loader:
                inputs, targets = data
                inputs, targets = inputs.to(device), targets.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                
                val_correct_predictions += accuracy(outputs, targets) * targets.size(0)
                val_total_predictions += targets.size(0)

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_accuracy = correct_predictions / total_predictions * 100
        val_accuracy = val_correct_predictions / val_total_predictions * 100

        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%')
            
        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), model_path)
            print(f"Saved best model to {model_path}")

        scheduler.step(val_loss)

        current_lr = scheduler.get_last_lr()[0] if hasattr(scheduler, 'get_last_lr') else optimizer.param_groups[0]['lr']
        print(f'Current lr: {current_lr:.6f}')


    print("Finished Training")  
    return model

trained_model = train_model(model, train_loader, val_loader, device)
        

  0%|          | 0/417 [00:00<?, ?it/s]

100%|██████████| 417/417 [01:02<00:00,  6.70it/s]


Epoch [1/150], Train Loss: 2.0545, Val Loss: 1.5396, Train Accuracy: 44.16%, Val Accuracy: 55.95%
Saved best model to CN_model=mobilenet_v2_dataset=CIFAR100_optim=Adam_scheduler=RoP_lr=0.001_factor=0.1_patience=5.pth
Current lr: 0.001000


100%|██████████| 417/417 [01:01<00:00,  6.76it/s]


Epoch [2/150], Train Loss: 1.3428, Val Loss: 1.2190, Train Accuracy: 60.53%, Val Accuracy: 63.95%
Saved best model to CN_model=mobilenet_v2_dataset=CIFAR100_optim=Adam_scheduler=RoP_lr=0.001_factor=0.1_patience=5.pth
Current lr: 0.001000


...

Epoch [140/150], Train Loss: 0.0087, Val Loss: 0.9971, Train Accuracy: 99.94%, Val Accuracy: 77.50%
Current lr: 0.000001


100%|██████████| 417/417 [01:01<00:00,  6.80it/s]


Epoch [141/150], Train Loss: 0.0086, Val Loss: 1.0029, Train Accuracy: 99.94%, Val Accuracy: 77.59%
Current lr: 0.000001


 44%|████▍     | 184/417 [00:27<00:34,  6.71it/s]


KeyboardInterrupt: 

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(m, data_loader, device):
    m.eval()  
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = m(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Print evaluation results
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

model.load_state_dict(torch.load(model_path, map_location=device)) # load saved parameters (weights and biases)
evaluate_model(model, test_loader, device)

/tmp/ipykernel_1318986/3855630808.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device)) # load saved parame

Accuracy: 0.7779
Precision: 0.7809
Recall: 0.7779
F1 Score: 0.7781
